In [4]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = (
                {'host':'192.168.250.10' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},
                {}
                
                               )

datalist=[]
 
for company in datacompany:
    msg_sql= """
                   
 declare @datefrom varchar(10), 
        @dateto varchar(10),
        @item varchar(10) ,
        @company varchar(20)

declare @table table ( docentry int ,transtype int , doc_ref varchar(50))


set @datefrom ='20220101'
set @dateto ='20220331'
set @company ='igu'
set @item = ''


insert into @table 
SELECT a.docentry ,a.objtype, c.beginstr + convert(varchar,b.docnum) SO FROM ODLN (nolock) A 
    INNER JOIN ORDR  (nolock) B ON A.u_IGU_SODOcEntry = b.docEntry
    inner join nnm1  (nolock) c on b.series = c.series 
WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto
union all 

SELECT a.docentry ,a.objtype, a.numatCard SO FROM OINV  (nolock) A 
WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto

union all
SELECT distinct a.docentry ,a.objtype,A.BASEREF FROM PDN1  (nolock) A 
    WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto AND A.BaseType=22


select * from 
(
select @company company,
        a.itemcode ,
       b.itemname ,
       ' Opening' transtype ,
       @datefrom docdate,
       '' doc_ref, 
       ' -' cardCode, 
       ' -'CardName ,
       ' -' ref1 ,
       ' -'ref2,
       sum(a.inqty ) in_quantity ,
       sum(a.outqty) out_quantity ,
       sum(a.inqty - a.outqty) quantity ,
       0 price,
	   sum((a.inqty - a.outqty)* a.calcprice) amount
        from OINM  (nolock) A 
    INNER JOIN OITM  (nolock) B ON A.ITEMCODE = B.ITEMCODE  
where convert(varchar,a.docdate,112) < @datefrom
and b.itemcode + b.itemname like '%' + isnull(@item,'') + '%'
group by a.itemcode ,
       b.itemname
union all 

select @company company,
        a.itemcode ,
       b.itemname ,
       c.name transtype ,
       convert(varchar,a.docdate,112) docdate,
       isnull(e.doc_ref,'') ,
       a.cardCode, 
       a.CardName ,
       a.ref1 ,
       a.ref2,
       a.inqty  quantity ,
        a.outqty quantity ,
       a.inqty - a.outqty quantity ,
       a.calcprice, 
	   (a.inqty - a.outqty)* a.calcprice amount       
        from OINM  (nolock) A 
    INNER JOIN OITM  (nolock) B ON A.ITEMCODE = B.ITEMCODE 
    left outer join [@igu_transType] c on a.transtype = c.code 
    left outer join @table e on a.transtype = e.transtype and a.createdby = e.docentry 

where convert(varchar, a.docdate,112) between @datefrom and @dateto
and b.itemcode + b.itemname like '%' + isnull(@item,'') + '%'
 
) as a 
order by company ,itemcode, docdate
 

"""

 
    

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df2 = pd.concat(datalist)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

df2

DatabaseError: Execution failed on sql '
                   
 declare @datefrom varchar(10), 
        @dateto varchar(10),
        @item varchar(10) ,
        @company varchar(20)

declare @table table ( docentry int ,transtype int , doc_ref varchar(50))


set @datefrom ='20210101'
set @dateto ='20210630'
set @company ='igu'
set @item = 'Seafood'


insert into @table 
SELECT a.docentry ,a.objtype, c.beginstr + convert(varchar,b.docnum) SO FROM ODLN (nolock) A 
    INNER JOIN ORDR  (nolock) B ON A.u_IGU_SODOcEntry = b.docEntry
    inner join nnm1  (nolock) c on b.series = c.series 
WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto
union all 

SELECT a.docentry ,a.objtype, a.numatCard SO FROM OINV  (nolock) A 
WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto

union all
SELECT distinct a.docentry ,a.objtype,A.BASEREF FROM PDN1  (nolock) A 
    WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto AND A.BaseType=22


select * from 
(
select @com

In [2]:
df2.to_excel (r'KStock 2022-Jan-Jun.xlsx', index=True, header=True, merge_cells=True)

In [ ]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = (
                {'host':'192.168.1.13' , 'dbname':'PTIMS','user':'sa','password':'B1admin'}
                               )

datalist=[]
 
for company in datacompany:
    msg_sql= """
                   
 select *
 
 from OITW
 

"""

In [ ]:
select  
        A.ITEMCODE ,
        B.ITEMNAME ,
        C.NAME TRANSNAME ,
        CONVERT(VARCHAR,A.DOCDATE,23) DOCDATE,
        A.DOCTIME,
        A.TransNum ,
        A.Ref1 ,
        A.Comments,
        A.InQty , 
        A.OutQty ,
        A.InQty- A.OutQty QUANTITY ,
        A.CalcPrice  ,
        a.TransValue
    FROM OINM A 
    INNER JOIN OITM B ON A.ITEMCODE = B.ITEMCODE
    INNER JOIN [@IGU_TRANSTYPE] C ON A.TRANSTYPE = C.CODE 
WHERE  convert(varchar,a.docdate,112) between '{{datefrom}}' and '{{dateto}}'
and a.TransType in (59,60)

In [ ]:
select  A.COUNTDATE,
    A.ITEMCODE ,
    B.ITEMNAME ,
    B.U_Group ,
    B.U_SubGroup ,
    B.InvntryUom ,
    A.CountQty ,
    A.CountQty -  A.InWhsQty as [difference],
    A.InWhsQty GENERAL_WAREHOUSE ,
    d.onhand VIRTUAL_WAREHOUSE ,
    c.AvgPrice
from inc1 a 
  inner join OITM B ON A.ITEMCODE  = B.ITEMCODE 
  INNER JOIN OITW C ON A.ITEMCODE = C.ITEMCODE AND C.WhsCode = A.WhsCode
  INNER JOIN
      (
        select itemcode , sum(onhand)onhand from oitw where whscode <>'01'
        group by itemcode 
      )D on a.itemcode = d.itemcode 
 WHERE A.LineStatus ='O'

In [14]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = (
                {'host':'192.168.250.10' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},
                
                               )

datalist=[]
 
for company in datacompany:
    msg_sql= """
                   
 select  A.COUNTDATE,
    A.ITEMCODE ,
    B.ITEMNAME ,
    B.U_Group ,
    B.U_SubGroup ,
    B.U_Spec,
    B.InvntryUom ,
    A.CountQty ,
    A.CountQty -  A.InWhsQty as [difference],
    A.InWhsQty GENERAL_WAREHOUSE ,
    d.onhand VIRTUAL_WAREHOUSE ,
    c.AvgPrice
from inc1 a 
  inner join OITM B ON A.ITEMCODE  = B.ITEMCODE 
  INNER JOIN OITW C ON A.ITEMCODE = C.ITEMCODE AND C.WhsCode = A.WhsCode
  INNER JOIN
      (
        select itemcode , sum(onhand)onhand from oitw where whscode <>'01'
        group by itemcode 
      )D on a.itemcode = d.itemcode 
 WHERE A.LineStatus ='O'
 

"""

 
    

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df2 = pd.concat(datalist)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

df2

,COUNTDATE,ITEMCODE,ITEMNAME,U_Group,U_SubGroup,U_Spec,InvntryUom,CountQty,difference,GENERAL_WAREHOUSE,VIRTUAL_WAREHOUSE,AvgPrice
0,2022-06-07,AAU1AMH002,AUS CHL S RUMP CAP IW VAC,A-MEAT,01-BEEF,128-Rump,KG,0.00,0.000,0.000,0.0,0.0000
1,2022-06-07,AAU1BOR003,AUS CHL PR 1601 OP RIBS,A-MEAT,01-BEEF,120-OP Ribs/Export Ribs,KG,0.00,0.000,0.000,0.0,215000.0000
2,2022-06-07,AAU1HRV022,AUS CHL A 2302 BOLAR BLADE IW VAC,A-MEAT,01-BEEF,133-Blade,KG,2033.19,50.970,1982.220,0.0,118000.0000
3,2022-06-07,AAU1HRV027,AUS CHL PR GF 2160 TENDERLOIN SS/OFF MSA BLACK...,A-MEAT,01-BEEF,102-Tenderloin,KG,602.85,-443.840,1046.690,0.0,458752.4021
4,2022-06-07,AAU1JBS017,"AUS CHL PR 2143 STRIPLOIN 5,0KG UP",A-MEAT,01-BEEF,104-Striploin,KG,0.00,0.000,0.000,0.0,118000.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
4754,2022-06-07,MAT3DBO013,DARBO MINIJARS-HONEY 60X28GM,E-GROCERY,28-SPREAD & JAM,60 x 28 GR,CTN,0.00,-101.947,101.947,0.0,272843.1683
4755,2022-06-07,MAT3DBO032,DARBO RASPBERRY SIEVED PRESERVE REFORM (DIABET...,M-CONFECTIONERY,None,None,JAR,0.00,0.000,0.000,0.0,23210.9376
4756,2022-06-07,MDE3BRG018,GOURMET MUESLI FRUITS 8 x 375GR,E-GROCERY,33-CEREAL,8 x 375 GR,BOX,0.00,0.000,0.000,0.0,33868.0354
4757,2022-06-07,OID2OLS004,PAKET COMBO SALMON,O-OTHERS,None,None,PKT,0.00,0.000,0.000,0.0,110500.0000


In [15]:
df2.to_excel (r'Stok opname 10Jun22.xlsx', index=True, header=True, merge_cells=True)

In [1]:
import pandas as pd
import pandas.io.sql
import pymssql as sapsql 
import numpy as np
from IPython.display import display
from jinja2 import Environment, FileSystemLoader
import pdfkit
import telegram 


env = Environment(loader=FileSystemLoader('/data/jupyter/'))
#template = env.get_template("template.html")

datacompany = (
                {'host':'192.168.250.10' , 'dbname':'IGU_LIVE','user':'sa','password':'B1admin'},
                
                               )

datalist=[]
 
for company in datacompany:
    msg_sql= """
                   
 declare @datefrom varchar(10), 
        @dateto varchar(10),
        @item varchar(10) ,
        @company varchar(20)

declare @table table ( docentry int ,transtype int , doc_ref varchar(50))


set @datefrom ='20220101'
set @dateto ='20220331'
set @company ='igu'
set @item = ''


insert into @table 
SELECT a.docentry ,a.objtype, c.beginstr + convert(varchar,b.docnum) SO FROM ODLN (nolock) A 
    INNER JOIN ORDR  (nolock) B ON A.u_IGU_SODOcEntry = b.docEntry
    inner join nnm1  (nolock) c on b.series = c.series 
WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto
union all 

SELECT a.docentry ,a.objtype, a.numatCard SO FROM OINV  (nolock) A 
WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto

union all
SELECT distinct a.docentry ,a.objtype,A.BASEREF FROM PDN1  (nolock) A 
    WHERE convert(varchar,a.docdate,112)between @datefrom and @dateto AND A.BaseType=22


select * from 
(
select @company company,
        a.itemcode ,
       b.itemname ,
       ' Opening' transtype ,
       @datefrom docdate,
       '' doc_ref, 
       ' -' cardCode, 
       ' -'CardName ,
       ' -' ref1 ,
       ' -'ref2,
       sum(a.inqty ) in_quantity ,
       sum(a.outqty) out_quantity ,
       sum(a.inqty - a.outqty) quantity ,
       0 price,
	   sum((a.inqty - a.outqty)* a.price*a.rate) amount
        from OIVL  (nolock) A 
    INNER JOIN OITM  (nolock) B ON A.ITEMCODE = B.ITEMCODE  
where convert(varchar,a.docdate,112) < @datefrom
and b.itemcode + b.itemname like '%' + isnull(@item,'') + '%'
group by a.itemcode ,
       b.itemname
union all 

select @company company,
        a.itemcode ,
       b.itemname ,
       c.name transtype ,
       convert(varchar,a.docdate,112) docdate,
       isnull(e.doc_ref,'') ,
       a.cardCode, 
       a.CardName ,
       a.ref1 ,
       a.ref2,
       a.inqty  quantity ,
        a.outqty quantity ,
       a.inqty - a.outqty quantity ,
       a.calcprice, 
	   (a.inqty - a.outqty)* a.calcprice amount       
        from OINM  (nolock) A 
    INNER JOIN OITM  (nolock) B ON A.ITEMCODE = B.ITEMCODE 
    left outer join [@igu_transType] c on a.transtype = c.code 
    left outer join @table e on a.transtype = e.transtype and a.createdby = e.docentry 

where convert(varchar, a.docdate,112) between @datefrom and @dateto
and b.itemcode + b.itemname like '%' + isnull(@item,'') + '%'
 
) as a 
order by company ,itemcode, docdate
 

"""

 
    

    conn = sapsql.connect(host=company["host"] , user=company["user"] , password=company["password"], database=company["dbname"] )
    cursor = conn.cursor()     
    data = pandas.io.sql.read_sql(msg_sql,conn)
    datalist.append(data)

df2 = pd.concat(datalist)  
options = {
            'page-size': 'legal',
            'orientation': 'landscape',
            }
#html_out = template.render(template_vars)
textstring = ""
#test = df.pivot_table(index=["groupname",'cardname'],columns=["PymntGroup"],aggfunc=np.sum,  values=["Balance"],fill_value="0",margins=True)

df2

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


,company,itemcode,itemname,transtype,docdate,doc_ref,cardCode,CardName,ref1,ref2,in_quantity,out_quantity,quantity,price,amount
0,igu,AAU1AAC001,AUS CHL WAG STRIPLOIN F1 MB 2-3,Opening,20220101,,-,-,-,-,10251.54,10251.54,0.0,0.0,0.0
1,igu,AAU1AAC002,AUS CHL WAG F1 TENDERLOIN MB 2-3,Opening,20220101,,-,-,-,-,4448.36,4448.36,0.0,0.0,0.0
2,igu,AAU1AAC003,AUS CHL WAG F1 STRIPLOIN MB 5,Opening,20220101,,-,-,-,-,5601.95,5601.95,0.0,0.0,0.0
3,igu,AAU1AAC004,AUS CHL WAG AA4 STRIPLOIN 1 RIB,Opening,20220101,,-,-,-,-,5270.37,5270.37,0.0,0.0,0.0
4,igu,AAU1AAC005,AUS CHL WAG ER 95CL,Opening,20220101,,-,-,-,-,3602.90,3602.90,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151576,igu,OID3LOC040,BOX PACKAGING TASTY FRIES UK 384X321X253,GOOD ISSUE,20220315,,1190012,PERSEDIAAN NON FOOD,2030000348,,0.00,72.00,-72.0,6300.0,-453600.0
151577,igu,OID3LOC040,BOX PACKAGING TASTY FRIES UK 384X321X253,GOOD ISSUE,20220315,,1190012,PERSEDIAAN NON FOOD,2030000344,,0.00,4.00,-4.0,6300.0,-25200.0
151578,igu,OID3LOC041,TRAY FOAM TGP 08 240X170X17MM,A/P INVOICE,20220308,,VL0278,"DWIPA MEGAH LESTARI, PT",2031000214,MC2030000110/INVDML/2203/0508,0.00,0.00,0.0,0.0,0.0
151579,igu,OID3LOC041,TRAY FOAM TGP 08 240X170X17MM,GOODS RECEIPT PO,20220308,2030000110,VL0278,"DWIPA MEGAH LESTARI, PT",2031000277,10547,10000.00,0.00,10000.0,345.0,3450000.0


In [2]:
df2.to_excel (r'KStock Jan-Mar22.xlsx', index=True, header=True, merge_cells=True)